In [1]:
import pandas as pd
import numpy as np

In [3]:
items = pd.read_csv('items.csv')
shops = pd.read_csv('shops.csv')
train = pd.read_csv('sales_train.csv.gz', compression='gzip', header=0, quotechar='"')
test = pd.read_csv('test.csv', header=0, quotechar='"')

In [71]:
item_categories = pd.read_csv('item_categories.csv', header=0, quotechar='"')

In [72]:
item_categories['cat-s'] = item_categories['item_category_name']

In [75]:
def mapping(df, names):
    for i in list(df['item_category_id']):
        if i == 0:
            continue
        elif i in range(1, 8):
            df.loc[i, 'cat-s'] = names[0]
        elif i in range(10, 18):
            df.loc[i, 'cat-s'] = names[1]
        elif i in range(18, 25):
            df.loc[i, 'cat-s'] = names[2]
        elif i in range(26, 28):
            df.loc[i, 'cat-s'] = names[3]
        elif i in range(28, 32):
            df.loc[i, 'cat-s'] = names[4]
        elif i in range(32, 37):
            df.loc[i, 'cat-s'] = names[5]
        elif i in range(37, 43):
            df.loc[i, 'cat-s'] = names[6]
        elif i in range(43, 55):
            df.loc[i, 'cat-s'] = names[7]
        elif i in range(55, 61):
            df.loc[i, 'cat-s'] = names[8]
        elif i in range(61, 73):
            df.loc[i, 'cat-s'] = names[9] 
        elif i in range(73, 79):
            df.loc[i, 'cat-s'] = names[10]
    return df

names = ['accessory', 'consoles', 'console games', 'phone games', 'CD games', 'card', 'movie', 'book', 'music', 'gift', 'soft']

In [76]:
item_categories = mapping(item_categories, names)

In [79]:
items = pd.merge(items, item_categories, on='item_category_id', how='left')
items.head()

,item_name,item_id,item_category_id,item_category_name,cat-s
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,Кино - DVD,movie
1,!ABBYY FineReader 12 Professional Edition Full...,1,76,Программы - Для дома и офиса (Цифра),soft
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,Кино - DVD,movie
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,Кино - DVD,movie
4,***КОРОБКА (СТЕКЛО) D,4,40,Кино - DVD,movie


In [80]:
set(train['shop_id'].unique()) > set(test['shop_id'].unique())

True

In [81]:
set(train['item_id'].unique()) > set(test['item_id'].unique())

False

In [82]:
train.shape

(2935849, 6)

In [86]:
train = train[(train['item_cnt_day'] < 1000) | (train['item_price'] < 100000)]
train.shape

(2935846, 6)

In [87]:
items = items[['item_id', 'cat-s']]

In [88]:
from itertools import product
index_cols = ['shop_id', 'item_id', 'date_block_num']
grid = []
for block_num in train['date_block_num'].unique():
    cur_shops = train.loc[train['date_block_num'] == block_num, 'shop_id'].unique()
    cur_items = train.loc[train['date_block_num'] == block_num, 'item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

In [90]:
train = train.groupby(['date_block_num','shop_id','item_id']).agg(
    {'item_cnt_day': np.sum, 'item_price': np.mean}).reset_index()

In [93]:
train.rename({'item_cnt_day': 'item_cnt_month'}, inplace=True)

In [94]:
train = pd.merge(grid, train, on=index_cols, how='left')

In [95]:
train = pd.merge(train, items, on=['item_id'], how='left')

In [96]:
test = pd.merge(test, items, on=['item_id'], how='left')

In [99]:
for type_ids in [['item_id'], ['shop_id'], ['cats'], ['item_id', 'shop_id']]:
    for column_id in ['item_price', 'item_cnt_month']:
        mean_df = train[type_ids + [column_id]].groupby(type_ids).agg(np.mean).reset_index()
        mean_df.rename(
            {column_id: "mean_of_"+column_id+"_groupby_"+"_".join(type_ids)}, inplace=True)
        
        train = pd.merge(train, mean_df, on=type_ids, how='left')
        test = pd.merge(test, mean_df, on=type_ids, how='left')

KeyError: "['item_price'] not in index"

In [100]:
train.head()

,shop_id,item_id,date_block_num,item_cnt_day,item_price_x,cat-s,item_price_y
0,59,22154,0,1.0,999.0,movie,696.576923
1,59,2552,0,NaN,NaN,music,942.750000
2,59,2554,0,NaN,NaN,music,1709.050000
3,59,2555,0,NaN,NaN,music,1124.466038
4,59,2564,0,NaN,NaN,music,339.073171
